# Fine-tuning the modified model with our data

## Imports


In [3]:
# connecting drive to colab notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import torchvision
import numpy as np
import torch
import argparse
import cv2
from PIL import Image
from google.colab.patches import cv2_imshow
import torch.nn as nn
from collections import OrderedDict
import torch.optim as optim
import time
import copy
import torchvision.transforms as transforms
from torchvision.io.image import read_image
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import sys
sys.path.append('/content/drive/MyDrive/drive_folder')
from custom_dataset_loader import TaiChiDataset, ToTensor

## Helper functions

In [7]:
# converting all the images to tensors and then normalize them
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

## Our prediction model

In [39]:
# Initialize model with the best available weights
# create an instance of (e.g.) torchvision.models.segmentation.fcn_resnet50
# and tell it to load pretrained weights
weights = FCN_ResNet50_Weights.DEFAULT
modified_model = fcn_resnet50(weights=weights)

# set computation device
device = torch.device('cuda')

# model to train() and load onto computation devicce
modified_model.to(device)

# we are feature extracting so we only need to compute weights for the new layer
set_parameter_requires_grad(modified_model, True)

In [ ]:
# modify that model by removing its final layer and replacing with a 2D vector output at each pixel(?) (instead of 20 class logits)

# (classifier): FCNHead(
#     (0): Conv2d(2048, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
#     (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#     (2): ReLU()
#     (3): Dropout(p=0.1, inplace=False)
#     (4): Conv2d(512, 21, kernel_size=(1, 1), stride=(1, 1))
#   )
# (aux_classifier): FCNHead(
#     (0): Conv2d(1024, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
#     (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#     (2): ReLU()
#     (3): Dropout(p=0.1, inplace=False)
#     (4): Conv2d(256, 21, kernel_size=(1, 1), stride=(1, 1))
#   )
# )

# instead of torch.Size([1, 21, 120, 240]) -> torch.Size([1, 2, 120, 240])
modified_model.classifier[3] = nn.Sequential()
modified_model.classifier[4] = nn.Conv2d(512, 2, kernel_size=(1, 1), stride=(1, 1))
# modified_model.aux_classifier[3] = nn.Sequential()
# modified_model.aux_classifier[4] = nn.Conv2d(256, 2, kernel_size=(1, 1), stride=(1, 1))
print(modified_model)

In [53]:
# see what parameters will be tuned
params_to_update = modified_model.parameters()
print('Params to learn:')
for name, param in modified_model.named_parameters():
  if param.requires_grad:
      print(name)

Params to learn:
classifier.4.weight
classifier.4.bias


### Train function

In [83]:
# define train function
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_aux=False):
  model.to(device)
  since = time.time()

  val_loss_history = []

  best_model_wts = copy.deepcopy(model.state_dict())

  for epoch in range(num_epochs):
        print('-' * 10)
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data.
            for i_batch, sample_batched in enumerate(dataloaders[phase]):
              inputs = sample_batched['image0']
              # print(inputs.shape)
              coords = sample_batched['coords']
              coords = coords.view(16,2,-1,1)
              # print(coords.shape)

              inputs = inputs.to(device).float() #torch.Size([16, 3, 120, 240])
              coords = coords.to(device) 
              # NEED TO RESHAPE COORDS ACCORDING TO MODEL OUTPUT:
              # torch.Size([16, 1, 4096, 2])-> torch.Size([16, 2, 120, 240])

              # zero the parameter gradients
              optimizer.zero_grad()

              # forward
              # track history if only in train
              with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    outputs = outputs['out'].view(16,2,-1,1) # torch.Size([16, 2, 120*240, 1])
                    # print(outputs.shape)
                    outputs = outputs[:, :, :coords.shape[2], :]
                    loss = criterion(outputs, coords)
                    
                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            print('{} Loss: {:.4f}'.format(phase, epoch_loss, ))

            # deep copy the model
            if epoch == 0:
              best_loss = epoch_loss
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_loss_history.append(epoch_loss)

            print()
            
        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('Best val loss: {:4f}\n'.format(best_loss))

  # load best model weights
  model.load_state_dict(best_model_wts)
  return model, val_loss_history




## Initializing Datasets and Dataloaders

In [ ]:
TRAIN_DATA = "training_data_2023-01-02"
!unzip -d "$TRAIN_DATA"/ /content/drive/MyDrive/"$TRAIN_DATA".zip # unziping training data

In [16]:
# Load dataset
dataset = TaiChiDataset(log_file=TRAIN_DATA+'/sample_ids.txt',
                        root_dir=TRAIN_DATA,
                        check=True,
                        transform=ToTensor()
                        )

# Split dataset into train and val
validation_split = .2
shuffle_dataset = True
random_seed= 42
batch_size = 16

dataset_size = len(dataset)
indices = list(range(dataset_size))

split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

# and load data
dataloaders_dict = {'train': train_loader, 'val': validation_loader}

## Training & evaluating the model

In [84]:
batch_size = 8
num_epochs = 15

# define optimizer
params_to_update = modified_model.parameters()
my_optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

# Setup the loss
my_criterion = nn.MSELoss()

# Train and evaluate
modified_model, hist = train_model(modified_model,
                                   dataloaders_dict,
                                   my_criterion,
                                   my_optimizer,
                                   num_epochs=num_epochs)

Epoch 0/14
----------
train Loss: 20045.7848

val Loss: 5001.8282

Training complete in 0m 40s
Best val loss: 5001.828156
Epoch 1/14
----------
train Loss: 19924.5036

val Loss: 4974.2205

Training complete in 1m 21s
Best val loss: 4974.220522
Epoch 2/14
----------
train Loss: 19851.6704

val Loss: 4959.2310

Training complete in 2m 3s
Best val loss: 4959.230974
Epoch 3/14
----------
train Loss: 19811.2705

val Loss: 4949.5878

Training complete in 2m 46s
Best val loss: 4949.587789
Epoch 4/14
----------
train Loss: 19777.3743

val Loss: 4942.8096

Training complete in 3m 29s
Best val loss: 4942.809641
Epoch 5/14
----------
train Loss: 19740.2344

val Loss: 4935.9177

Training complete in 4m 12s
Best val loss: 4935.917716
Epoch 6/14
----------


KeyboardInterrupt: ignored